In [2]:
import pandas as pd

In [3]:
from pyspark import HiveContext,SQLContext

In [4]:
from pyspark.sql import SparkSession

In [5]:
from pyspark.sql.types import *

In [36]:
from pyspark.sql.window import Window

In [37]:
from pyspark.sql.functions import col, row_number, concat

In [6]:
spark = SparkSession.builder.appName("Test Dataframes Converter").enableHiveSupport().getOrCreate()

In [7]:
hive = HiveContext(spark)

In [8]:
sql_Ctx = SQLContext(spark)

Junção de 2 listas

In [9]:
list = [['OK','OK','NOK'],['table1','table2','table3'],['','oi oi oi','ciao ciao ciao']]

In [66]:
list_seq = [1,2,3]

In [68]:
list2 = list.append(list_seq)

In [72]:
list

[['OK', 'OK', 'NOK'],
 ['table1', 'table2', 'table3'],
 ['', 'oi oi oi', 'ciao ciao ciao'],
 [1, 2, 3]]

In [73]:
headers = ['status','tabela','desc','seq']

In [74]:
df_pd = pd.DataFrame.from_dict(dict(zip(headers,list)))

In [75]:
df_pd

,status,tabela,desc,seq
0,OK,table1,,1
1,OK,table2,oi oi oi,2
2,NOK,table3,ciao ciao ciao,3


In [76]:
type(df_pd)

pandas.core.frame.DataFrame

In [77]:
spark_df = sql_Ctx.createDataFrame(df_pd).sort("tabela")

In [78]:
spark_df.show()

+------+------+--------------+---+
|status|tabela|          desc|seq|
+------+------+--------------+---+
|    OK|table1|              |  1|
|    OK|table2|      oi oi oi|  2|
|   NOK|table3|ciao ciao ciao|  3|
+------+------+--------------+---+



------------------------------------

Incluir numero de linha

In [47]:
w = Window().orderBy("tabela")

In [53]:
spark_df.withColumn("rowID", row_number().over(w)).show()

+------+------+--------------+-----+
|status|tabela|          desc|rowID|
+------+------+--------------+-----+
|   NOK|table3|ciao ciao ciao|    1|
|    OK|table1|              |    2|
|    OK|table2|      oi oi oi|    3|
+------+------+--------------+-----+



------------------------------------

In [80]:
spark_df.count()

3

In [81]:
spark_df.show()

+------+------+--------------+---+
|status|tabela|          desc|seq|
+------+------+--------------+---+
|    OK|table1|              |  1|
|    OK|table2|      oi oi oi|  2|
|   NOK|table3|ciao ciao ciao|  3|
+------+------+--------------+---+



In [83]:
spark_df.registerTempTable("tb_error_list")

In [84]:
sql_Ctx.sql("""select * from tb_error_list""").show()

+------+------+--------------+---+
|status|tabela|          desc|seq|
+------+------+--------------+---+
|    OK|table1|              |  1|
|    OK|table2|      oi oi oi|  2|
|   NOK|table3|ciao ciao ciao|  3|
+------+------+--------------+---+



In [20]:
sql_Ctx.sql("""select current_timestamp() from tb_error_list""").show(truncate=False)

+-----------------------+
|current_timestamp()    |
+-----------------------+
|2019-06-05 14:01:47.295|
|2019-06-05 14:01:47.295|
|2019-06-05 14:01:47.295|
+-----------------------+

